# 📊 ИССЛЕДОВАНИЕ ДАННЫХ
## Генетическая оптимизация торговых индикаторов

**Цель:** Изучить структуру данных `df_btc_15m_complete.csv`, проверить качество данных, проанализировать распределения и доступные колонки.

**Дата:** 2025-11-10  
**Таймфрейм:** 15 минут  
**Источник:** `dataframe/with_full_depth/df_btc_15m_complete.csv`


In [ ]:
# 📦 ИМПОРТЫ

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Добавляем путь к src для импорта модулей
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root / "src"))

from data_loader import DataLoader

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Импорты выполнены!")
print(f"📁 Рабочая директория: {Path().resolve()}")


In [ ]:
# 📁 ЗАГРУЗКА ДАННЫХ

# Путь к данным
data_path = project_root.parent / "dataframe" / "with_full_depth" / "df_btc_15m_complete.csv"

print(f"🔍 Проверяем файл: {data_path}")
print(f"   Существует: {data_path.exists()}")

# Загрузка данных
loader = DataLoader(data_path=str(data_path))
df = loader.load_data()

print(f"\n✅ Данные загружены!")
print(f"📊 Размер: {df.shape}")
print(f"📅 Период: {df.index.min()} - {df.index.max()}")


In [ ]:
# 📋 СТРУКТУРА ДАННЫХ

print("="*60)
print("📋 СТРУКТУРА ДАННЫХ")
print("="*60)

print(f"\n📊 Колонки ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    missing = df[col].isnull().sum()
    missing_pct = (missing / len(df)) * 100
    print(f"   {i:2d}. {col:25s} | {str(dtype):15s} | Пропусков: {missing:6d} ({missing_pct:5.2f}%)")

print(f"\n📈 Первые 5 строк:")
df.head()


In [ ]:
# 📊 СТАТИСТИКА ПО КОЛОНКАМ

print("="*60)
print("📊 СТАТИСТИКА ПО ОСНОВНЫМ КОЛОНКАМ")
print("="*60)

# Основные ценовые данные
price_cols = ['open', 'high', 'low', 'close']
if all(col in df.columns for col in price_cols):
    print("\n💰 ЦЕНОВЫЕ ДАННЫЕ (OHLC):")
    print(df[price_cols].describe())
    
# Объемные данные
volume_cols = ['volume', 'flow_buy_volume', 'flow_sell_volume', 'flow_trade_volume', 'flow_trade_imbalance']
volume_cols = [col for col in volume_cols if col in df.columns]
if volume_cols:
    print("\n📊 ОБЪЕМНЫЕ ДАННЫЕ:")
    print(df[volume_cols].describe())
    
# Order Book данные
ob_cols = [col for col in df.columns if any(x in col.lower() for x in ['ask', 'bid', 'd3', 'd5', 'd8', 'd60', 'ratio', 'diff'])]
if ob_cols:
    print(f"\n📚 ORDER BOOK ДАННЫЕ ({len(ob_cols)} колонок):")
    print(df[ob_cols].describe())


In [ ]:
# 🔍 ПРОВЕРКА КАЧЕСТВА ДАННЫХ

print("="*60)
print("🔍 ПРОВЕРКА КАЧЕСТВА ДАННЫХ")
print("="*60)

# Проверка пропусков
print("\n1️⃣ ПРОВЕРКА ПРОПУСКОВ:")
missing_summary = df.isnull().sum()
missing_cols = missing_summary[missing_summary > 0]
if len(missing_cols) > 0:
    print("   ⚠️ Обнаружены пропуски:")
    for col, count in missing_cols.items():
        pct = (count / len(df)) * 100
        print(f"      {col}: {count} ({pct:.2f}%)")
else:
    print("   ✅ Пропусков не обнаружено!")

# Проверка дубликатов
print("\n2️⃣ ПРОВЕРКА ДУБЛИКАТОВ:")
duplicates = df.index.duplicated().sum()
if duplicates > 0:
    print(f"   ⚠️ Обнаружено {duplicates} дубликатов индексов")
else:
    print("   ✅ Дубликатов не обнаружено")

# Проверка временных интервалов
print("\n3️⃣ ПРОВЕРКА ВРЕМЕННЫХ ИНТЕРВАЛОВ:")
time_diffs = df.index.to_series().diff()
expected_interval = pd.Timedelta(minutes=15)
actual_intervals = time_diffs.value_counts().head(10)
print(f"   Ожидаемый интервал: {expected_interval}")
print(f"   Фактические интервалы (топ-10):")
for interval, count in actual_intervals.items():
    print(f"      {interval}: {count} раз")


In [ ]:
# 📈 ВИЗУАЛИЗАЦИЯ ЦЕНОВЫХ ДАННЫХ

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('📈 АНАЛИЗ ЦЕНОВЫХ ДАННЫХ (BTC 15m)', fontsize=16, fontweight='bold')

# График цены закрытия
axes[0, 0].plot(df.index, df['close'], linewidth=0.8, alpha=0.8)
axes[0, 0].set_title('Цена закрытия (Close)')
axes[0, 0].set_xlabel('Дата')
axes[0, 0].set_ylabel('Цена (USDT)')
axes[0, 0].grid(True, alpha=0.3)

# Распределение цен
axes[0, 1].hist(df['close'], bins=100, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Распределение цен закрытия')
axes[0, 1].set_xlabel('Цена (USDT)')
axes[0, 1].set_ylabel('Частота')
axes[0, 1].grid(True, alpha=0.3)

# Объемы торгов
if 'volume' in df.columns:
    axes[1, 0].plot(df.index, df['volume'], linewidth=0.5, alpha=0.6, color='orange')
    axes[1, 0].set_title('Объем торгов')
    axes[1, 0].set_xlabel('Дата')
    axes[1, 0].set_ylabel('Объем')
    axes[1, 0].grid(True, alpha=0.3)

# Волатильность (изменения цены)
price_changes = df['close'].pct_change() * 100
axes[1, 1].plot(df.index, price_changes, linewidth=0.3, alpha=0.6, color='red')
axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=0.8)
axes[1, 1].set_title('Изменения цены (%)')
axes[1, 1].set_xlabel('Дата')
axes[1, 1].set_ylabel('Изменение (%)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Графики созданы!")


In [ ]:
# 📊 АНАЛИЗ ORDER BOOK ДАННЫХ

ob_cols = [col for col in df.columns if any(x in col.lower() for x in ['ask', 'bid', 'd3', 'd5', 'd8', 'd60', 'ratio', 'diff'])]

if ob_cols:
    print("="*60)
    print("📊 АНАЛИЗ ORDER BOOK ДАННЫХ")
    print("="*60)
    
    print(f"\n📋 Найдено {len(ob_cols)} колонок с Order Book данными:")
    for col in sorted(ob_cols):
        print(f"   - {col}")
    
    # Визуализация дисбалансов
    imbalance_cols = [col for col in ob_cols if col.startswith('d')]
    if imbalance_cols:
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle('📊 ДИСБАЛАНСЫ ORDER BOOK', fontsize=16, fontweight='bold')
        
        for idx, col in enumerate(imbalance_cols[:4]):
            ax = axes[idx // 2, idx % 2]
            ax.plot(df.index, df[col], linewidth=0.5, alpha=0.6)
            ax.axhline(y=0, color='red', linestyle='--', linewidth=0.8)
            ax.set_title(f'{col}')
            ax.set_xlabel('Дата')
            ax.set_ylabel('Дисбаланс')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    # Статистика по дисбалансам
    if imbalance_cols:
        print("\n📈 Статистика по дисбалансам:")
        print(df[imbalance_cols].describe())
else:
    print("⚠️ Order Book данные не найдены")


In [ ]:
# 📊 АНАЛИЗ FLOW ДАННЫХ

flow_cols = [col for col in df.columns if 'flow' in col.lower()]

if flow_cols:
    print("="*60)
    print("📊 АНАЛИЗ FLOW ДАННЫХ")
    print("="*60)
    
    print(f"\n📋 Найдено {len(flow_cols)} колонок с Flow данными:")
    for col in flow_cols:
        print(f"   - {col}")
    
    # Визуализация flow данных
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('📊 FLOW ДАННЫЕ (Потоки покупок/продаж)', fontsize=16, fontweight='bold')
    
    for idx, col in enumerate(flow_cols[:4]):
        ax = axes[idx // 2, idx % 2]
        ax.plot(df.index, df[col], linewidth=0.5, alpha=0.6)
        ax.set_title(f'{col}')
        ax.set_xlabel('Дата')
        ax.set_ylabel('Объем')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Статистика
    print("\n📈 Статистика по Flow данным:")
    print(df[flow_cols].describe())
    
    # Анализ дисбаланса
    if 'flow_trade_imbalance' in df.columns:
        print("\n📊 Анализ дисбаланса торгов:")
        imbalance = df['flow_trade_imbalance']
        print(f"   Средний дисбаланс: {imbalance.mean():.2f}")
        print(f"   Медианный дисбаланс: {imbalance.median():.2f}")
        print(f"   Стандартное отклонение: {imbalance.std():.2f}")
else:
    print("⚠️ Flow данные не найдены")


In [ ]:
# 📊 РАЗДЕЛЕНИЕ ДАННЫХ НА TRAIN/VAL/TEST

print("="*60)
print("📊 РАЗДЕЛЕНИЕ ДАННЫХ")
print("="*60)

train, val, test = loader.split_data(train_split=0.7, val_split=0.15, test_split=0.15)

print("\n✅ Данные разделены на:")
print(f"   Train:  {len(train)} записей ({len(train)/len(df)*100:.1f}%)")
print(f"   Val:    {len(val)} записей ({len(val)/len(df)*100:.1f}%)")
print(f"   Test:   {len(test)} записей ({len(test)/len(df)*100:.1f}%)")

# Визуализация разделения
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(train.index, train['close'], label='Train', linewidth=0.8, alpha=0.7, color='blue')
ax.plot(val.index, val['close'], label='Validation', linewidth=0.8, alpha=0.7, color='orange')
ax.plot(test.index, test['close'], label='Test', linewidth=0.8, alpha=0.7, color='green')
ax.set_title('📊 Разделение данных на Train/Validation/Test', fontsize=14, fontweight='bold')
ax.set_xlabel('Дата')
ax.set_ylabel('Цена закрытия (USDT)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 📝 ВЫВОДЫ

### ✅ Качество данных:
- [x] **Проверка пропусков** - выполнена в ячейке 5, обнаружены колонки с пропусками (если есть)
- [x] **Проверка дубликатов** - выполнена в ячейке 5, проверка дубликатов индексов
- [x] **Проверка временных интервалов** - выполнена в ячейке 5, анализ фактических интервалов между записями

### 📊 Доступные данные:
- [x] **Ценовые данные (OHLC)** - `open`, `high`, `low`, `close` присутствуют в данных
- [x] **Объемные данные** - `volume`, `flow_buy_volume`, `flow_sell_volume`, `flow_trade_volume`, `flow_trade_imbalance`
- [x] **Order Book данные** - колонки с префиксами `d3`, `d5`, `d8`, `d60`, `ask`, `bid`, `ratio`, `diff`
- [x] **Flow данные** - колонки с `flow` в названии для анализа потоков покупок/продаж

### 📈 Результаты анализа:
- **Структура данных:** Изучена в ячейке 3 - количество колонок, типы данных, пропуски
- **Статистика:** Собрана в ячейке 4 - описательная статистика по всем группам данных
- **Визуализация:** Создана в ячейках 6-8 - графики цен, объемов, Order Book и Flow данных
- **Разделение данных:** Выполнено в ячейке 9 - Train (70%) / Validation (15%) / Test (15%)

### 🎯 Следующие шаги:
1. ✅ **Анализ доступных индикаторов** из `08_indicator_engine_clean.ipynb` - содержит 43+ технических индикаторов
2. **Выбор финального списка индикаторов** для оптимизации - определить приоритетные индикаторы
3. **Подготовка данных для генетического алгоритма** - интеграция данных с системой оптимизации
4. **Настройка параметров генетического алгоритма** - определение пространства поиска параметров
5. **Запуск оптимизации** - выполнение генетического алгоритма для выбранных индикаторов
